# Generator

Notebook uses Large Language Models to generate the text for 10 authors from `res/selected/author_list` file. It uses the list of queries from `res/queries` and the system specification message that together create the prompt. The goal is to generate text in writing style of the specific author, where the topic is of secondary importance. Results are saved in `res` directory.

In order to run the notebook create `.env` file in the root of the project and set the api keys in the following manner:

```bash
ANTHROPIC_API_KEY=YOUR_API_KEY
GOOGLE_API_KEY=YOUR_API_KEY
MISTRAL_API_KEY=YOUR_API_KEY
OPENAI_API_KEY=YOUR_API_KEY
```

Below there is a list of used LLMs along with their default settings that are used. The intention is to give the LLM the freedom, without many constraints, in generation process. Default settings provide sufficient level of such flexibility to the model.

### OpenAI defaults

[07.06.2024] [Reference](https://platform.openai.com/docs/api-reference/chat)

```json
{
  "seed": null,
  "temperature": 1.0,
  "top_p": 1.0,
  "frequency_penalty": 0.0,
  "presence_penalty": 0.0,
  "n": 1,
  "stop": null,
  "echo": false,
  "best_of": 1,
}
```

### Gemini defaults

[07.06.2024] [Reference](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini#request_body)

```json
{
  "temperature": 1.0,
  "topK": 32,
  "topP": 1,
  "frequencyPenalty": 0,
  "presencePenalty": 0,
  "stopSequences": null,
}
```

### MistralAI defaults

[07.06.2024] [Reference](https://docs.mistral.ai/api/#operation/createChatCompletion)

```json
{
  "temperature": 0.7,
  "top_p": 1,
  "random_seed": null
}
```
### Anthropic defaults

[07.06.2024] [Reference](https://docs.anthropic.com/en/api/messages)

```json
{
  "stop_sequences": null,
  "temperature": 1.0,
}
```

Max token parameter is set to 4096, due to the contraints of `claude-3-haiku-20240307` model.



In [90]:
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_openai import ChatOpenAI

from src import *

In [91]:
secrets = Secrets()
configuration = Configuration()

In [92]:
models = {
    "gpt-3.5-turbo-0125": ChatOpenAI(model="gpt-3.5-turbo-0125", max_tokens=configuration.max_tokens),
    "gpt-4o": ChatOpenAI(model="gpt-4o", api_key=secrets.openai_api_key, max_tokens=configuration.max_tokens),
    "gemini-1.5-flash": ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=secrets.google_api_key, max_tokens=configuration.max_tokens),
    "open-mixtral-8x7b": ChatMistralAI(model="open-mixtral-8x7b", api_key=secrets.mistral_api_key, max_tokens=configuration.max_tokens),
    "claude-3-haiku-20240307": ChatAnthropic(model="claude-3-haiku-20240307", api_key=secrets.anthropic_api_key, max_tokens=configuration.max_tokens),
}

In [93]:
text_generator = TextGenerator(
    models=models,
    queries_path=configuration.query_filepath,
    authors_path=configuration.selected_authors_filepath,
    res_directory=configuration.res_directory,
    response_number_of_words=configuration.response_number_of_words
)

In [94]:
generated_texts, unprocessed_requests = text_generator.generate()

[1/125] Generating [gpt-3.5-turbo-0125]-[Lucy Maud Montgomery] Describe a dream you might have after a particularly spicy dinner.
[2/125] Generating [gpt-4o]-[Lucy Maud Montgomery] Describe a dream you might have after a particularly spicy dinner.
[3/125] Generating [gemini-1.5-flash]-[Lucy Maud Montgomery] Describe a dream you might have after a particularly spicy dinner.
[4/125] Generating [open-mixtral-8x7b]-[Lucy Maud Montgomery] Describe a dream you might have after a particularly spicy dinner.
[5/125] Generating [claude-3-haiku-20240307]-[Lucy Maud Montgomery] Describe a dream you might have after a particularly spicy dinner.
[6/125] Generating [gpt-3.5-turbo-0125]-[Lucy Maud Montgomery] Convince a group of fish that life on land is far superior.
[7/125] Generating [gpt-4o]-[Lucy Maud Montgomery] Convince a group of fish that life on land is far superior.
[8/125] Generating [gemini-1.5-flash]-[Lucy Maud Montgomery] Convince a group of fish that life on land is far superior.
[9/12

In [95]:
unprocessed_requests

[]